In [1]:
# !pip install squarify

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import pandas as pd
import numpy as np
import re
from underthesea import word_tokenize
from gensim import corpora, models, similarities
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def df_RMF_preprocessing(df):
    # drop rows where Quantity < 0
    df = df[df.Quantity >= 0]
    # drop rows where CustomerID == null
    df = df[df.CustomerID.notnull()]
    # drop rows where UnitPrice < 0
    df = df[df.UnitPrice >= 0]
    # drop duplicated rows
    df = df.drop_duplicates()
    # Convert column 'InvoiceDate' to datetime datatype
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    # convert InvoiceNo data type to integer
    df.InvoiceNo = df.InvoiceNo.astype(int)
    # Create new column 'Amount'
    df['Amount'] = df['Quantity'] * df['UnitPrice']
    # Get max date of dataframe
    max_date = df['InvoiceDate'].max().date()
    # Calculate R, F, M
    Recency = lambda x: (max_date - x.max().date()).days
    Frequency = lambda x: x.nunique()
    Monetary = lambda x: round(sum(x),2)

    df_RFM = df.groupby('CustomerID').agg({'InvoiceDate' : Recency,
                                        'InvoiceNo' : Frequency,
                                        'Amount' : Monetary,
                                        })
    # Rename column names
    df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
    df_RFM = df_RFM.sort_values('Monetary', ascending=False)
    return df_RFM

In [4]:
# Calculate average values for each GMM_segment, and return a size of each segment 
def create_df_agg(df, groupby_col):
    df.R = df.R.astype(int)
    df.F = df.F.astype(int)
    df.M = df.M.astype(int)
    
    df_agg = df.groupby(groupby_col).agg({
      'Recency': 'mean',
      'Frequency': 'mean',
      'Monetary': 'mean',
      'R': 'mean',
      'F': 'mean',
      'M': ['mean', 'count']}).round(0)

    df_agg.columns = df_agg.columns.droplevel()
    df_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'RMean','FMean','MMean','Count']
    df_agg['Percent'] = round((df_agg['Count']/df_agg.Count.sum())*100, 2)

    # Reset the index
    df_agg = df_agg.reset_index()
    return df_agg

In [5]:
# Visualization - Treemap
def treemap_customer_segmentation(df_agg,font_size):
    #Create our plot and resize it.
    fig = plt.gcf()
    ax = fig.add_subplot()
    fig.set_size_inches(22, 9)

    # create a color palette, mapped to these values
    cmap = matplotlib.cm.plasma
    norm = matplotlib.colors.Normalize(vmin=min(df_agg['Count']), vmax=max(df_agg['Count']))
    colors = [cmap(norm(value)) for value in df_agg['Count']]

    squarify.plot(sizes=df_agg['Count'],
                text_kwargs={'fontsize':font_size,'weight':'bold', 'fontname':"sans serif"},
                color=colors,
                label=['{} \n{:.0f} days \n{:.0f} orders \n{:.0f} $ \n{:.0f} customers ({}%)'.format(*df_agg.iloc[i])
                        for i in range(0, len(df_agg))], alpha=0.5 )


    plt.title("Customers Segments",fontsize=26,fontweight="bold")
    plt.axis('off')

    # plt.savefig('RFM Segments.png')
    plt.show()

In [6]:
def univariate_cont_analysis(vars,df):
    results = []
    for var in vars:
        mean = df[var].mean()
        median = df[var].median() 
        mode = df[var].mode()
        max_val = df[var].max()
        min_val = df[var].min()
        range_val = np.ptp(df[var])
        variance = df[var].var()
        skewness = df[var].skew()
        kurtosis = df[var].kurtosis()
        result = [var, mean,median,mode,max_val,min_val,range_val,variance,skewness,kurtosis]
        results.append(result)
        df_result = pd.DataFrame(results, columns=['var_name','mean','median','mode','max_val','min_val',
                                                  'range_val','variance','skewness','kurtosis'],
                                ).set_index('var_name')
    return df_result

In [7]:
#Visualization - distplot
def visualization_distplot(vars,df):
    plt.figure(figsize=(15,4))
    for i in range(1,len(vars)+1):
        plt.subplot(1,3,i)
        sns.distplot(df[vars[i-1]].dropna())
    plt.tight_layout()
    plt.show()

In [8]:
#Visualization - boxplot
def visualization_boxplot(vars,df):
    plt.figure(figsize=(15,4))
    for i in range(1,len(vars)+1):
        plt.subplot(1,3,i)
        plt.boxplot(df[vars[i-1]].dropna())
        plt.xlabel(str(vars[i-1]))
    plt.tight_layout()
    plt.show()

In [9]:
def clean_content(text, list_to_remove):
    #xoá bỏ ký tự đặc biệt
    text = re.sub('[\-\–\,\+\?\%\/\•\*\&\[\]\(\)\:\;]', ' ', text).replace('v.',' ').replace('...',' ').replace('.',' ').replace('…',' ')
    #lowercase và tách riêng các câu phân cách bởi '\n'
    text = text.lower().split('\n')
    #loại bỏ khoảng trắng ở đầu và cuối mỗi câu
    text = [e.strip() for e in text]
    #loại bỏ các câu trùng nhau
    text = list(set(text) - set(list_to_remove))
    #loại bỏ các câu chứa 'sku'
    text = [e for e in text if 'sku' not in e]
    return text

In [10]:
def gensim_preparation(clean_df,stop_words):
    # Tokenize(split) sentence into words
    products_gen = [[text for text in x.split()] for x in clean_df['content_wt']]
    
    # remove some more special elements & stopwords 
    products_gen_re = [[t.lower() for t in text if not t in ['', ' ','thương hiệu','xuất xứ']] for text in  products_gen] # ký tự đặc biệt
    products_gen_re = [[t for t in text if not t in stop_words] for text in products_gen_re] # stopword
    
    # Obtain the number of features based on dicjtionary: use corpora.Dictionary
    dictionary = corpora.Dictionary(products_gen_re)
    
    # Numbers of features (word) in dictionary
    feature_cnt = len(dictionary.token2id)
    
    # Obtain corpus based on dictionary (dense matrix)
    corpus = [dictionary.doc2bow(text) for text in products_gen_re]
    
    # Use TF-IDF to process corpus, obtaining index
    tfidf = models.TfidfModel(corpus)

    # Tính toán sự tương tự trong ma trận thưa thớt
    index_sparse_matrix = similarities.SparseMatrixSimilarity(tfidf[corpus],
                                               num_features=feature_cnt)
    return tfidf, dictionary, index_sparse_matrix

In [11]:
# When user choose one product
def get_recommendation(clean_df, original_df, tfidf, dictionary, index_sparse_matrix, i):
    """
    Gợi ý 5 sản phẩm có độ tương đồng cao nhất 
    """
    # sản phẩm đang xem
    name_description_pre = clean_df.iloc[[i]]['content_wt'][i]
    view_product = name_description_pre.lower().split()

    # convert search words into Sparse Vectors
    kw_vector = dictionary.doc2bow(view_product)

    # similarity calculation
    sim = index_sparse_matrix[tfidf[kw_vector]]
    
    # 5 sản phẩm có sim cao nhất (không tính sản phẩm chọn có index -1)
    indexes = sim.argsort()[-6:-1]
    result = original_df[['item_id', 'name', 'description', 'rating', 'price', 'list_price', 'brand', 'group','url']].iloc[sim.argsort()[-6:-1]]
    print('*** SELECTED PRODUCT ***')
    display(original_df[['item_id', 'name', 'description', 'rating', 'price', 'list_price', 'brand', 'group','url']].iloc[[i]])
    print('*** 5 RECOMMENDED PRODUCTS ***')
    return result

In [12]:
def cosine_function(original_df, stop_words):
    """
    Dùng hàm "TfidfVectorizer" để chuẩn hóa "content_wt" với:
    + analyzer='word': chọn đơn vị trích xuất là word
    + ngram_range=(1, 1): mỗi lần trích xuất 1 word
    + min_df=0: tỉ lệ word không đọc được là 0
    Lúc này ma trận trả về với số dòng tương ứng với số sp và số cột tương ứng với số từ được tách ra từ "content_wt"
    """
    tf = TfidfVectorizer(analyzer='word', stop_words=stop_words, min_df=0)
    tfidf_matrix = tf.fit_transform(original_df['content_wt'])

    """
    Dùng hàm "linear_kernel" để tạo thành ma trận hình vuông với số hàng và số cột là số lượng sp
    để tính toán điểm tương đồng giữa từng sp với nhau
    """
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [13]:
# với mỗi sản phẩm, lấy 5 sản phẩm tương quan nhất
def cosine_recommendation(original_df,cosine_similarities,i):
    results = {}
    for idx, row in original_df.iterrows():    
        similar_indices = cosine_similarities[idx].argsort()[-6:-1]
        similar_items = [(cosine_similarities[idx][e]) for e in similar_indices]
        similar_items = [(cosine_similarities[idx][e], original_df.index[e]) for e in similar_indices]
#         print(similar_items[0:])
        results[idx] = similar_items[0:]
    print('*** SELECTED PRODUCT ***')
    display(original_df[['item_id', 'name', 'description', 'rating', 'price', 'list_price', 'brand', 'group','url']].iloc[[i]])
    print('*** 5 RECOMMENDED PRODUCTS ***')
    return original_df[['item_id', 'name', 'description', 'rating', 'price', 'list_price', 'brand', 'group','url']].iloc[[e[1] for e in results[i]]]